# Paseando con la Encuesta CASEN

En esta clase cargaremos un dataset _real_, que es utilizado tanto para la toma de decisiones y políticas públicas, como para investigación. La encuesta de caracterización socio-económica nacional, CASEN, en el año 2015. La encuesta CASEN se hace cada dos años en todo el país. Actualmente los datos de la versión 2017 se están procesando y debiesen estar disponibles en algunos meses.

Al terminar esta clase debiésemos haber aprendido a:

  * Cargar e inspeccionar un dataset utilizando `pandas`.
  * Visualización exploratoria de variables, utilizando `pandas`, `seaborn` y `matplotlib`.
  * Comprender que no siempre los datos se pueden analizar de manera directa, fila por fila (u observación por observación), sino que existen restricciones que impiden hacer cálculos representativos sin tener que utilizar operaciones auxiliares.
  * Calcular distribuciones de variables a través de la agrupación de observaciones.
  * Calcular distribuciones representativas para la población.
  
Necesitaremos también hacer uso de los siguientes archivos:

  * [Libro de Códigos CASEN 2015](http://observatorio.ministeriodesarrollosocial.gob.cl/casen-multidimensional/casen/docs/Libro_de_Codigos_Casen_2015.pdf)

## Preámbulo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(context='poster', style='ticks', palette='Accent', font='Linux Biolinum O', font_scale=1.1)
%matplotlib inline

## Datasets

In [ ]:
casen_path = '../udd/datasets/2015_casen_survey/casen2015.csv.gz'
names_path = '../udd/datasets/2015_casen_survey/diccionario_dpa.xlsx.csv'

In [ ]:
comunal_names = pd.read_csv(names_path, index_col='cut_comuna').rename(columns={'nombre_comuna_dl2339': 'nombre_comuna'})
comunal_names.head()

In [ ]:
casen = (pd.read_csv(casen_path, sep=',', encoding='iso-8859-1')
         .join(comunal_names.nombre_comuna, on='comuna')
        )

casen.info()

In [ ]:
casen.head()

In [ ]:
list(casen.columns)

### Exploración de Variables

Una manera típica de explorar variables es a través de histogramas, que nos permiten ver la frecuencia con la que aparecen en el dataset valores distribuidos en cajas o _bins_.

In [ ]:
casen['edad'].plot(kind='hist', bins=10)
sns.despine()

In [ ]:
sns.countplot(casen['sexo'])

In [ ]:
sns.countplot(casen['region'])
sns.despine()

**Ejercicio**: elegir una variable para cada tipo de gráfico y probarlo.

----

El módulo `seaborn` tiene funciones que permiten hacer diagramas más informativos. Por ejemplo, el "distribution plot" mezcla un histograma (al que se le calcula automáticamente un número óptimo de _bins_) junto a una _Kernel Density Estimation_, que permite ver la función que genera el histograma:

In [ ]:
sns.distplot(casen['edad'])
#sns.despine()

Y, aunque no es el foco de la clase de hoy, es posible también usar `distplot` en una grilla de gráficos a través del objeto `sns.FacetGrid`:

In [ ]:
sns.FacetGrid(data=casen, col='zona', hue='sexo', size=6, aspect=1.5).map(sns.distplot, 'edad').add_legend()

¿Cómo interpretan estos resultados?

**Ejercicio**: ¿Qué combinaciones de variables se les ocurren?

---

Ahora veremos los "factores de expansión" de la encuesta. Cada observación (que corresponde a una persona) tiene un _ponderador_ que indica a cuántas otras personas representa. Entonces, cuando dije que no se podían realizar análisis de manera directa, es porque _no todas las personas tienen el mismo peso dentro de la encuesta_.

Hay varios factores de expansión, pero nos enfocaremos en el factor regional: `expr`.

In [ ]:
casen['expr'].describe()

In [ ]:
sns.distplot(casen['expr'])
plt.xlim([0, 1000])

Como pueden ver, una persona como mínimo se representa solamente a sí misma, mientras que hay otras que cuentan por más de cuatro mil.

**Ejercicio**: Para facilitar el análisis, trabajaremos solamente con datos de la región metropolitana. ¿Cómo hacerlo?

In [ ]:
casen_rm = ???
casen_rm.shape

Hay 267 mil personas en la encuesta, pero representan a...

In [ ]:
casen_rm['expr'].sum()

Veamos cuántas observaciones hay por comuna:

In [ ]:
plt.figure(figsize=(8,14))
sns.countplot(y='nombre_comuna', data=casen_rm)
sns.despine()

**Ejercicio**: ¿Cómo podemos calcular la población total por comuna representada por la encuesta?

In [ ]:
population = ???

plt.figure(figsize=(8,14))
sns.barplot(y='nombre_comuna', x='expr', data=population)
sns.despine()

## Tipos de Preguntas

En la encuesta hay por lo menos tres tipos de preguntas:

  * Las que tienen una respuesta cuantitativa. Por ej., _¿Cuál es tu ingreso?¿Cuántos años tienes?_
  * Las que tienen respuestas categóricas. Por ej., _¿Cuál es tu sexo?_
  * Las que tienen text. Por ej., _¿En qué comuna vivías hace cinco años?_
  
Como vimos, estimar las distribuciones de los dos primeros tipos no es algo directo debido a los factores de expansión. ¡No considerar los factores de expansión introducirá errores en los resultados!

## Preguntas con Respuestas Categóricas

Probemos la pregunta `o25c` (miren el libro de códigos) que trata sobre el medio de transporte utilizado para ir al trabajo.

Primero, vemos que no todas las personas responden esta pregunta:

In [ ]:
casen_rm['o25c'].sample(5)

**¿Por qué?**

---

Las respuestas son numéricas:

In [ ]:
sns.countplot(y='o25c', data=casen_rm)
sns.despine()

Podemos manualmente asignar etiquetas, que transcribimos desde el libro de códigos:

In [ ]:
labels = ['Transporte Público', 'Motorizado Particular', 'A Pie', 'Bicicleta u otro No Motorizado', 'Otro']
sns.countplot(y='o25c', data=casen_rm)
plt.gca().set_yticklabels(labels)
sns.despine()

Haremos una función para calcular el total expandido de un dataframe:

In [ ]:
variable = 'o25c'

def expanded_total(dataframe):
    return (dataframe.groupby(variable)
            .aggregate({'expr': 'sum'})
            .reset_index()
    )

expanded_total(casen_rm)

Pero el total no es necesariamente informativo. Ahora revisaremos cómo podemos calcular las estadísticas a nivel comunal.

Necesitamos es poder calcular el total de una población por comuna.

In [ ]:
relevant_population = (casen_rm[~pd.isnull(casen_rm[variable])]
                       .groupby('nombre_comuna')
                       .aggregate({'expr': 'sum'})
)

relevant_population.sample(5)

También necesitaremos una manera de calcular el total expandido para cada alternativa de la pregunta:

In [ ]:
expanded_values = (casen_rm[~pd.isnull(casen_rm[variable])]
                   .groupby('nombre_comuna')
                   .apply(expanded_total)
)

expanded_values

In [ ]:
values_per_municipality = (expanded_values.reset_index()
                           .pivot_table(index='nombre_comuna', columns='o25c', values='expr')
                           .fillna(0.0))
values_per_municipality

In [ ]:
fractions_per_municipality = values_per_municipality.div(relevant_population['expr'], axis='index')
fractions_per_municipality

In [ ]:
fractions_per_municipality.columns = labels

In [ ]:
fractions_per_municipality

In [ ]:
plt.figure(figsize=(8,24))
sns.heatmap(fractions_per_municipality, annot=True, linewidth=1)

In [ ]:
sns.clustermap(fractions_per_municipality, annot=True, linewidth=1, figsize=(8,24))

Ahora haremos una función que automatice el análisis que realizamos:

In [ ]:
def matrix_sum(source, labels, group_col, test_var, test_exp):
    source_population = source[~pd.isnull(source[test_var])].groupby(group_col).agg({test_exp: 'sum'})

    test_agg = (source[~pd.isnull(source[test_var])].groupby(group_col)
        .apply(lambda x: x.groupby(test_var).agg({test_exp: 'sum'}).reset_index())
        .reset_index()
        .pipe(lambda x: x.pivot_table(index=group_col, columns=test_var, values=test_exp))
        .fillna(0.0)
        .pipe(lambda x: x.div(source_population[test_exp], axis='index'))
    )
    
    test_agg.columns = labels
    
    return test_agg

fractions_per_municipality = matrix_sum(casen_rm, labels, 'nombre_comuna', 'o25c', 'expr')
fractions_per_municipality.sample(5)

In [ ]:
plt.figure(figsize=(6,24))
sns.heatmap(fractions_per_municipality, 
            fmt='.2f', xticklabels=labels, annot=True, cmap='PuBu', center=0.5, cbar=False, linewidth=1)

Usemos nuestra función para explorar otra pregunta: `oficio1`.

In [ ]:
labels = [
    'Fuerzas Armadas',
    'Poder Ejecutivo/Legislativo',
    'Profesionales, Científicos e Intelectuales',
    'Técnicos Profesionales de nivel medio',
    'Empleados de Oficina',
    'Servicios y Vendedores',
    'Calificados (agricultura, agropecuaria)',
    'Oficiales y operarios de artes mecánicas',
    'Operadores e instaladores de maquinaria',
    'No calificados'
]

fractions_per_municipality = matrix_sum(casen_rm, labels, 'nombre_comuna', 'oficio1', 'expr')

In [ ]:
sns.clustermap(fractions_per_municipality.T, row_cluster=False, metric='cosine', method='ward', annot=True,
               fmt='.2f', figsize=(36,8), cmap='viridis', cbar=False)

## Datos Cuantitativos

In [ ]:
from statsmodels.stats.weightstats import DescrStatsW

In [ ]:
def weighted_stats(g):
    d = DescrStatsW(g.ytotcorh.values, weights=g['expr'].values)
    quantiles = d.quantile([0.025, 0.5, 0.975], return_pandas=False)
    return pd.DataFrame.from_records([{'mean_ytotcorh': d.mean, 
                                       'std_ytotcorh': d.std,
                                       'std_var': d.var,
                                       'percentile_025_ytotcorh': quantiles[0], 
                                       'percentile_50_ytotcorh': quantiles[1],
                                       'percentile_975_ytotcorh': quantiles[2]}])

In [ ]:
test = (casen_rm.drop_duplicates(subset='folio')
            .groupby('nombre_comuna')
            .apply(weighted_stats)
            .pipe(lambda x: x.reset_index().drop('level_1', axis=1).set_index(group_col))
           )
test_agg

In [ ]:
plt.figure(figsize=(12,6))
plt.errorbar(np.arange(1, test_agg.shape[0] + 1), test_agg.mean_ytotcorh,  
         yerr=[test_agg.mean_ytotcorh - test_agg['percentile_025_ytotcorh'], 
               test_agg['percentile_975_ytotcorh'] - test_agg.mean_ytotcorh],
         linestyle='None', marker='o')
plt.gca().ticklabel_format(style='plain')
plt.xticks(np.arange(1, test_agg.shape[0] + 1), test_agg.index.values, rotation=90)
plt.ylabel('Stuff')
sns.despine()
#plt.scatter(np.arange(1, test_agg.shape[0] + 1), test_agg.percentile_50_ytotcorh, marker='x')